# 实验1

In [4]:
class Byte:
    def __init__(self, x=0):
        self.val = x & 0xff

    def update(self):
        self.val &= 0xff

    def get(self):
        self.update()
        return self.val

class Bit32:
    def __init__(self, x=0):
        self.val = x & 0xffffffff

    def update(self):
        self.val = self.val & 0xffffffff

    def get(self):
        self.update()
        return self.val

    def __setitem__(self, index, value):
        if value:
            self.val = self.val | (value << index)
        else:
            self.val = self.val & ~(value << index)
        self.update()

    def __getitem__(self, index):
        self.update()
        if self.val & (1 << index):
            return 1
        else:
            return 0

class Registers:
    def __init__(self, n):
        self.files = [Bit32() for _ in range(n)]
        self.size = n

    def __getitem__(self, item):
        return self.files[item]

    def __setitem__(self, key, value):
        self.files[key] = value

    def output(self):
        for i in range(self.size):
            print("$%d:\t0x%016x\t%d" % (i, self.files[i].val, self.files[i].val))
            


class Registers_file:
    def __init__(self, n=32):
        self.files = Registers(32)

    def ports_out(self, a_addr, b_addr):
        a_out = Bit32(0) if not a_addr else self.files[a_addr]
        b_out = Bit32(0) if not b_addr else self.files[b_addr]
        return a_out, b_out

    def port_single_out(self, addr) -> Bit32:
        return Bit32(0) if not addr else self.files[addr]

    def ports_in(self, addr, c_in: Bit32):
        if addr:
            self.files[addr] = c_in

    def output(self):
        self.files.output()
        

In [5]:
def byte_to_bit16(byte1, byte0):
    return (byte1 << 8) | byte0

def bit16_to_byte(bits):
    byte1 = (bits >> 8) & 0xff
    byte0 = bits & 0xff
    return byte1, byte0


def byte_to_bit32(byte3, byte2, byte1, byte0):
    bit16_1 = byte_to_bit16(byte3, byte2)
    bit16_0 = byte_to_bit16(byte1, byte0)
    return (bit16_1 << 16) | bit16_0


def bit32_to_byte(bits):
    bit16_1 = (bits >> 16) & 0xffff
    bit16_0 = bits & 0xffff
    byte3, byte2 = bit16_to_byte(bit16_1)
    byte1, byte0 = bit16_to_byte(bit16_0)
    return byte3, byte2, byte1, byte0

class Memory:
    def __init__(self, size):
        self.files = [Byte() for _ in range(size)]
        self.size = size

    def data_32_port_out(self, addr) -> Bit32:
        byte0, byte1, byte2, byte3 = self.files[addr].val, self.files[addr + 1].val, \
                                     self.files[addr + 2].val, self.files[addr + 3].val
        return Bit32(byte_to_bit32(byte3, byte2, byte1, byte0))

    def data_32_port_in(self, bits: Bit32, addr):
        byte3, byte2, byte1, byte0 = bit32_to_byte(bits.get())
        self.files[addr] = Byte(byte0)
        self.files[addr + 1] = Byte(byte1)
        self.files[addr + 2] = Byte(byte2)
        self.files[addr + 3] = Byte(byte3)
    
    def force_set(self, byte: Byte, addr: int):
        self.files[addr] = byte

    def output(self):
        for i in range(0x1000, 0x1000 + 64, 4):
            print("0x%08x:" % i, end=" ")
            for j in range(4):
                print("%02x" % self.files[i + j].val, end="\t")
            print("")
            
    def output_range(self, low, high):
        for i in range(low, high, 4):
            print("0x%08x:" % i, end=" ")
            for j in range(4):
                print("%02x" % self.files[i + j].val, end="\t")
            print("")


In [6]:
from enum import Enum, IntEnum

class InstructionId(Enum):
    Load = 0
    Store = 1
    Add = 2


Register = IntEnum("Register", ['R0','R1','R2','R3','R4','R5','R6','R7','R8'
                                ,'R9','R10','R11','R12','R13','R14','R15',
                                'R16','R17','R18','R19','R20','R21','R22',
                                'R23','R24','R25','R26','R27','R28','R29',
                                'R30','R31'], start = 0)

class Instruction:
    def __init__(self, id: InstructionId) -> None:
        self.id = id
    
    def __str__(self) -> str:
        return str(self.id)[14:]
    
    def execute(self, rf: Registers_file, mem: Memory):
        return

class Load(Instruction):
    def __init__(self, src: Register, addr: int) -> None:
        super().__init__(InstructionId.Load)
        self.src = src
        self.addr = addr
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "r{}".format(self.src) + ", " + "#{}".format(self.addr)
    
    def execute(self, rf: Registers_file, mem: Memory):
        data = mem.data_32_port_out(self.addr)
        rf.ports_in(self.src, data)
    
class Store(Instruction):
    def __init__(self, dst: Register, addr: int) -> None:
        super().__init__(InstructionId.Store)
        self.dst = dst
        self.addr = addr
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "r{}".format(self.dst) + ", " + "#{}".format(self.addr)
    
    def execute(self, rf: Registers_file, mem: Memory):
        data = rf.port_single_out(self.dst)
        mem.data_32_port_in(data, self.addr)
    
class Add(Instruction):
    def __init__(self, src1: Register, src2: Register, dst: Register) -> None:
        super().__init__(InstructionId.Add)
        self.src1 = src1
        self.src2 = src2
        self.dst = dst
    
    def __str__(self) -> str:
        return super().__str__() + "\t" + "r{}".format(self.dst) + ", " + "r{}".format(self.src1) + ", " + "r{}".format(self.src2)
    
    def execute(self, rf: Registers_file, mem: Memory):
        src1_data, src2_data = rf.ports_out(self.src1, self.src2)
        dst_data = Bit32(src1_data.get() + src2_data.get())
        rf.ports_in(self.dst, dst_data)
        

def parser(asm: str) -> list[Instruction]:
    
    def get_number(s: str) -> int:
        return int("".join(list(filter(lambda s: s.isdigit(), list(s)))))
    
    res: list[Instruction] = []
    instr_list = list(filter(lambda s: len(s) > 0, asm.split("\n")))
    for instr in instr_list:
        tokens = instr.split()
        # TODO It need Python 3.10
        match tokens:
            case ["Load", dst_s, addr_s]:
                dst: Register = Register.R0 + get_number(dst_s) # type: ignore
                addr = get_number(addr_s)
                res.append(Load(dst, addr))
            case ["Store", src_s, addr_s]:
                src: Register = Register.R0 + get_number(src_s) # type: ignore
                addr = get_number(addr_s)
                res.append(Store(src, addr))
            case ["Add", dst_s, src1_s, src2_s]:
                src1: Register = Register.R0 + get_number(src1_s) # type: ignore
                src2: Register = Register.R0 + get_number(src2_s) # type: ignore
                dst: Register = Register.R0 + get_number(dst_s) # type: ignore
                res.append(Add(src1, src2, dst))
        # HELP If Python < 3.10, uncomment it
        # if tokens[0] == "Load":
        #     dst: Register = Register.R0 + get_number(tokens[1]) # type: ignore
        #     addr = get_number(tokens[2])
        #     res.append(Load(dst, addr))
        # elif tokens[0] == "Store":
        #     src: Register = Register.R0 + get_number(tokens[1]) # type: ignore
        #     addr = get_number(tokens[2])
        #     res.append(Store(src, addr))
        # elif tokens[0] == "Add":
        #     dst: Register = Register.R0 + get_number(tokens[1]) # type: ignore
        #     src1: Register = Register.R0 + get_number(tokens[2]) # type: ignore
        #     src2: Register = Register.R0 + get_number(tokens[3]) # type: ignore
        #     res.append(Add(src1, src2, dst))
    
    return res

s = '''
Load r1, #0
Load r2, #1
Add r3, r1, r2
Store r3, #3
'''    

rf = Registers_file()
# 地址空间太大，仅模拟0 ~ 1024的地址
mem = Memory(1024)
mem.force_set(Byte(0x11), 0)
mem.force_set(Byte(0x22), 1)
mem.force_set(Byte(0x33), 2)
mem.force_set(Byte(0x44), 3)
mem.force_set(Byte(0x55), 4)

mem.output_range(0, 8)



instrs = parser(s)
for instr in instrs:
    instr.execute(rf, mem)

rf.output()
mem.output_range(0, 8)
    


0x00000000: 11	22	33	44	
0x00000004: 55	00	00	00	
$0:	0x0000000000000000	0
$1:	0x0000000044332211	1144201745
$2:	0x0000000055443322	1430532898
$3:	0x0000000099775533	2574734643
$4:	0x0000000000000000	0
$5:	0x0000000000000000	0
$6:	0x0000000000000000	0
$7:	0x0000000000000000	0
$8:	0x0000000000000000	0
$9:	0x0000000000000000	0
$10:	0x0000000000000000	0
$11:	0x0000000000000000	0
$12:	0x0000000000000000	0
$13:	0x0000000000000000	0
$14:	0x0000000000000000	0
$15:	0x0000000000000000	0
$16:	0x0000000000000000	0
$17:	0x0000000000000000	0
$18:	0x0000000000000000	0
$19:	0x0000000000000000	0
$20:	0x0000000000000000	0
$21:	0x0000000000000000	0
$22:	0x0000000000000000	0
$23:	0x0000000000000000	0
$24:	0x0000000000000000	0
$25:	0x0000000000000000	0
$26:	0x0000000000000000	0
$27:	0x0000000000000000	0
$28:	0x0000000000000000	0
$29:	0x0000000000000000	0
$30:	0x0000000000000000	0
$31:	0x0000000000000000	0
0x00000000: 11	22	33	33	
0x00000004: 55	77	99	00	
